Ophalen van startdatum van grondwater data

In [ ]:
# Hier gebruik ik de url van de standalone webservice 
# Te vinden in het f12 menu -> m -> start embeded tomcat service
base_url = "http://localhost:8080/FewsWebServices/rest/fewspiservice/v1/" 

In [ ]:
endpoint = "timeseries?"

# pas waar nodig aan
params = dict(
    filterId="grondwater_ruw",
    parameterIds="Stijghoogte.ongevalideerd",
    locationIds="B45C1081", # om te testen met 1 locatie
    startTime="1990-01-01T00:00:00Z",
    endTime="2025-01-01T00:00:00Z",
    onlyHeaders="true",
    showStatistics="true",
    documentFormat="PI_JSON"
)

In [ ]:
import requests
url = base_url + endpoint
res = requests.get(url, params=params)

In [4]:
data = res.json()
data

{'version': '1.34',
 'timeZone': '0.0',
 'timeSeries': [{'header': {'type': 'instantaneous',
    'moduleInstanceId': 'ImportKeller',
    'locationId': 'B45C1081',
    'parameterId': 'Stijghoogte.ongevalideerd',
    'timeStep': {'unit': 'second', 'multiplier': '3600'},
    'startDate': {'date': '1990-01-01', 'time': '00:00:00'},
    'endDate': {'date': '2025-01-01', 'time': '00:00:00'},
    'missVal': '-999.0',
    'stationName': 'Haa_3017_01',
    'lat': '51.60541326356878',
    'lon': '5.176536036944744',
    'x': '140406.0',
    'y': '401857.0',
    'z': '0.0',
    'units': 'm NAP',
    'firstValueTime': {'date': '2010-03-10', 'time': '15:00:00'},
    'lastValueTime': {'date': '2025-01-01', 'time': '00:00:00'},
    'maxValue': '8.674',
    'minValue': '4.805',
    'valueCount': '120002'}}]}

In [5]:
# voor als je de bovenste cellen niet kan runnen, hier alsnog de data
import json
with open('data.json','w')as f_out:
    f_out.write(json.dumps(data,indent=4))

with open('data.json','r')as f_in:
    data = json.load(f_in)

In [6]:
import pandas as pd
df = pd.json_normalize(data['timeSeries'],sep="_")
cols = df.columns
df.rename(columns=dict(zip(cols,('_'.join(col.split('_')[1:]) for col in cols))), inplace=True)
df = df.drop(columns=['type', 'moduleInstanceId', 'timeStep_unit', 'timeStep_multiplier', 'startDate_date',
       'startDate_time', 'endDate_date', 'endDate_time', 'missVal', 'units', 'lat', 'lon', 
       'lastValueTime_date', 'lastValueTime_time' # zelfde als lastvalue
       ])

df_sub = df.dropna(subset="firstValueTime_date").copy()
df_sub['firstValueTime'] = df_sub.apply(lambda x: pd.Timestamp(str(x.firstValueTime_date) +" "+ str(x.firstValueTime_time)),axis=1)
df_sub = df_sub.drop(columns=['firstValueTime_date', 'firstValueTime_time'])

In [7]:
df_sub.to_csv("data_output_nl.csv",sep=";",decimal=',')
df_sub.to_csv("data_output.csv")